# Making Cube images

This notebook makes cubes out of '.fits' images and their PSF' of 5 different bands obtained by the Hyper Suprime Cam (HSC) Telescope in order to use them along with Scarlet package and same images from the HST telscope for the purpose of separating (deblending) hosting Galaxies from Quasars

In [1]:
# import numpy as np
import scarlet
import scarlet.display 
from scarlet.display import AsinhMapping
from scarlet.initialization import build_initialization_coadd

from astropy.wcs import WCS
from astropy.nddata import Cutout2D


import pylab as plt
import astropy.io.fits as fits
from astropy.io import fits as pf
from astropy.visualization import astropy_mpl_style
import os, re

%matplotlib inline

plt.style.use(astropy_mpl_style)

import matplotlib
import matplotlib.pyplot as plt

# use a better colormap and don't interpolate the pixels
matplotlib.rc('image', cmap='gist_stern')
matplotlib.rc('image', interpolation='none')

from astropy.nddata import Cutout2D
import warnings
warnings.simplefilter("ignore")


### 1. HSC Cube

In [2]:
#location of the HSC images
path = './data/HSC_Images/'


#sort HSC fits files based on their starting name index in an ordered list
images = []
wcss = []
# We use images' statring name (in our case it start with a number from 2 up 70) to loop through the different bands
# Each 5 bands image has the same staring number, the only difference is the band name (G, I, R, Y, Z)
for i in range (2, len(os.listdir(path))+1):
    for image in os.listdir(path):
        if re.match(str(i)+'-', image):
            hdus = fits.open(path + image)[1]
            # WCS assign sky coordinates to the files in order to get their location and match it later with HST images
            wcs = WCS(hdus.header)
            img = hdus.data
            ny, nx = img.shape
            #Cutout2D is for the purpose of reducing the orignal size of images, we take the center of the image as a reference
            cut = Cutout2D(img, (ny//2, nx//2), (50, 50), wcs = wcs)
            images.append(cut.data)
            wcss.append(cut.wcs)
#             plt.imshow(img)
#             plt.show()

# creat a folder to save the HSC_cube images 
path_hsc_cube = '/./data/HSC_cube'
    
# save the hdu's in an array of size 5 each (i.e. based on the number of bands)

ind = 2
for i in range(0, len(images), 5):
    hdu = fits.PrimaryHDU(images[i : i+5], wcss[i].to_header())
    hdul = pf.HDUList([hdu])
    
    #name the newly formed cube images, and then save them to a new folder
#     hdul.writeto(path_hsc_cube + f'/{ind}-cube_hsc_image.fits', clobber=True)
    ind += 1


#     asinh= scarlet.display.AsinhMapping(minimum=img.min(), stretch=1, Q=1)
#     img_rgb= scarlet.display.img_to_rgb(images[i], norm=asinh)

#     plt.imshow(img_rgb)
#     plt.show()


OSError: [Errno 24] Too many open files: './data/HSC_Images/41-cutout-HSC-R-9813-pdr2_dud.fits'

### 2. HSC PSF Cube

In [3]:
path = './data/HSC_PSF/'
path_cube_hsc_psf = './data/HSC_PSF_cube'

#sort HSC PSF fits files based on their starting name index in an ordered list
images = []

for i in range (2, len(os.listdir(path))+1):
    for image in os.listdir(path):
        if re.match(str(i)+'-', image):
            hdus = fits.open(path + image)[0]
            img = hdus.data
            ny, nx = img.shape
            cut = Cutout2D(img, (ny//2, nx//2), (40, 40), wcs = None)
            images.append(cut.data)
#             plt.imshow(cut.data)
#             plt.show()
            
ind = 2
for i in range(0, len(images), 5):
    
    hdu = fits.PrimaryHDU(images[i : i+5])
    hdul = pf.HDUList([hdu])
    
    hdul.writeto(path_cube_hsc_psf + f'/{ind}-cube_hsc_psf.fits', clobber=True)
    ind += 1


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
  File "<ipython-input-3-29830c9896ac>", line 10, in <module>
    hdus = fits.open(path + image)[0]
  File "/anaconda3/lib/python3.7/site-packages/astropy/io/fits/hdu/hdulist.py", line 151, in fitsopen
  File "/anaconda3/lib/python3.7/site-packages/astropy/io/fits/hdu/hdulist.py", line 390, in fromfile
  File "/anaconda3/lib/python3.7/site-packages/astropy/io/fits/hdu/hdulist.py", line 1039, in _readfrom
  File "/anaconda3/lib/python3.7/site-packages/astropy/utils/decorators.py", line 521, in wrapper
  File "/anaconda3/lib/python3.7/site-packages/astropy/io/fits/file.py", line 178, in __init__
  File "/anaconda3/lib/python3.7/site-packages/astropy/io/fits/file.py", line 547, in _open_filename
  File "/anaconda3/lib/python3.7/site-packages/astropy/io/fits/util.py", line 397, in fileobj_open
OSError: [Errno 24] Too many open files: './data/HSC_PSF/41

OSError: [Errno 24] Too many open files: './data/HSC_PSF/41-psf-calexp-pdr2_dud-HSC-I-9813-2,2-150.58136-1.92691.fits'

### 3. HST Cut
This code will cut the HST images in order to match the size of HSC images, HST are almost 5 times higher in resolution than HSC images.

In [ ]:
path = './data/acs_mosaic_2.0/'

images = []
wcss = []

for i in range (2, len(os.listdir(path))+1):
    for image in os.listdir(path):
        if re.match(str(i)+'_', image):
            hdus = fits.open(path + image)
            wcs = WCS(hdus[0].header)
            img = hdus[0].data
            ny, nx = img.shape
            cut = Cutout2D(img, (ny//2, nx//2), (125, 125), wcs = wcs)
            images.append(cut.data)
            wcss.append(cut.wcs)
#             plt.imshow(img)
#             plt.show()

# # creat a folder to save the HST_cut images 
path_hst_cut = './data/HST_CUT'
    

ind = 2
for i in range(0, len(images)):
    
    hdu = fits.PrimaryHDU(images[i], wcss[i].to_header())

    hdul = pf.HDUList([hdu])
    hdul.writeto(path_hst_cut + f'/{ind}-hst_cut_cosmos.fits', clobber=True)
    ind += 1


#     asinh= scarlet.display.AsinhMapping(minimum=img.min(), stretch=1, Q=1)
#     img_rgb= scarlet.display.img_to_rgb(images[i], norm=asinh)

#     plt.imshow(img_rgb)
#     plt.show()
